## Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import gensim
import logging
import re
import nltk.data
from nltk.corpus import stopwords
import pysentiment as ps
from nltk.stem.porter import *
from __future__ import division
from sklearn.feature_extraction.text import TfidfVectorizer
import xgboost as xgb
import codecs
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
import sys
reload(sys)
sys.setdefaultencoding('utf-8')
stemmer = PorterStemmer() 


## Stopwords List

In [ ]:
stops = set(stopwords.words("english"))


## Lexicon Features Extraction Function

In [245]:
## Reading Lexicons
stock_lex = pd.read_csv('stock_lex.csv',sep=',')
def split_words(x):
    x =  re.sub("[^a-zA-Z]"," ", x)
    s = x.split()
    words = [w for w in s if len(w) > 1]
    return ' '.join(words)
stock_lex['cleanItem'] = stock_lex['Item'].apply(split_words)
stock_lex = stock_lex[stock_lex['cleanItem'] != '']

In [246]:
stock_lex.shape

(20336, 5)

In [247]:
## Function to compute lexicon scores
def compute_stock_score(text):
    words = nltk.word_tokenize(text)
    tagged = nltk.pos_tag(words)
    pos_score = 0
    neg_score = 0
    for i in range(0,len(tagged)):
        word = tagged[i]
        print word
        if word[1] != 'NNP':
            for index,row in stock_lex.iterrows():
                if word[0] == row['cleanItem'] and word[1] == row['POS']:
                    print row['cleanItem']
                    pos_score = pos_score+row['Aff_Score']
                    neg_score = neg_score+row['Neg_Score']
                    break
                else:
                    if i > 0:
                        prev = tagged[i-1]
                        words = row['cleanItem'].split()
                        if len(words)>1:
                            if prev[0] == words[0] and word[0] == words[1]:
                                print row['cleanItem']
                                pos_score = pos_score+row['Aff_Score']
                                neg_score = neg_score+row['Neg_Score']
                                break
    return pos_score,neg_score

In [249]:
# Computing Lexicon Scores Training Data
pos_score = []
neg_score = []
for index,row in training_data.iterrows():
    print index
    p,n = compute_stock_score(row['merged_spans'])
    pos_score.append(p)
    neg_score.append(n)
training_data['stockLex_Pos'] = pos_score
training_data['stockLex_Neg'] = neg_score


In [250]:
# Computing Lexicon Scores Test Data
pos_score = []
neg_score = []
for index,row in test_data.iterrows():
    print index
    p,n = compute_stock_score(row['merged_spans'])
    pos_score.append(p)
    neg_score.append(n)
test_data['stockLex_Pos'] = pos_score
test_data['stockLex_Neg'] = neg_score


## Reading Microblogs Dataset

In [134]:
training_data = pd.read_json(codecs.open('semeval-2017-task-5-subtask-1/Microblogs_Trainingdata_withmessages.json', 'r', 'utf-8'),orient='records')


In [135]:
test_data = pd.read_json(codecs.open('semeval-2017-task-5-subtask-1/Microblogs_Testdata_withmessages.json', 'r', 'utf-8'),orient='records')


In [136]:
training_data.shape

(1694, 6)

In [137]:
test_data.shape

(799, 5)

In [138]:
## Test data with empty spans
test_empty_span = test_data[test_data['spans'] == '']

In [139]:
test_empty_span.shape

(10, 5)

In [140]:
print test_empty_span.iloc[0]['text']

In [141]:
test_empty_span

,cashtag,id,source,spans,text
26,$AMZN,719796218071621600,twitter,,Commented on Apple In Need Of Innovation: Sell...
219,$GOOG,13088382,stocktwits,,Published $GOOG estimate on Estimize: +0.03 EP...
279,$FB,719525463878557700,twitter,,$FB 21% decline in original posts https://t.co...
418,$TZA,5670556,stocktwits,,Bot $TZA 31.30
632,$AAPL,6206304,stocktwits,,RT EXPstocktrader: Do your homework on $AAPL c...
670,$GDX,32370562,stocktwits,,$GDX $GDXJ $NUGT $DUST $JNUG $JDST Gold Miners...
691,$INFI,719903541167464400,twitter,,AAPL Day Low\nSee Here ---> https://t.co/en3DK...
716,$FB,719564967972765700,twitter,,STO Apr $FB 109/112 call spreads at 1.40.
728,$CHTR,710110155216584700,twitter,,Charter $CHTR Stock Gains Ahead of FCC Approva...
748,$TNET,711745824884187100,twitter,,Nice names on watch for tomorrow $TUES long $8...


In [142]:
training_data = training_data[['id','cashtag', 'sentiment score', 'spans', 'text']]

In [143]:
training_data.shape

(1694, 5)

## Preprocessing Text

In [164]:
def cleanText(text):
    print str(text).encode("utf-8")
    text = str(text).encode("utf-8")
    newtext = ' '.join(re.sub("(@[A-Za-z]+)|([^A-Za-z \t])|(\w+:\/\/\S+)"," ",text).lower().split())
    newt = newtext.split()
    words = [w for w in newt if len(w) > 2]
    return ' '.join(words)

In [165]:
def merge_spans(train):
        merged_spans = []
        for index,row in train.iterrows():
            if row['spans'][0] != '':
                print row['spans']
                merged_spans.append(cleanText(" ".join(row['spans'])))
            else:
                print row['spans']
                merged_spans.append(cleanText(row['text']))
        return merged_spans
        

In [166]:
def merge_spans_test(test):
        merged_spans = []
        for index,row in test.iterrows():
            if row['spans'] != '':
                print 'yo'
                merged_spans.append(cleanText(row['spans']))
            else:
                print 'no'
                merged_spans.append(cleanText(row['text']))
        return merged_spans
        

In [167]:
# Merging Spans
training_data['merged_spans'] = merge_spans(training_data)

In [168]:
training_data

,id,cashtag,sentiment score,spans,text,merged_spans
0,719659409228451840,$FB,0.366,[watching for bounce tomorrow],$FB watching for bounce tomorrow,watching for bounce tomorrow
1,719904304207962112,$LUV,0.638,[record number of passengers served in 2015],record number of passengers served in 2015 #ai...,record number passengers served
2,5329774,$NFLX,-0.494,[out $NFLX -.35],out $NFLX -.35,out nflx
3,719891468173844480,$DIA,0.460,"[Looking for a strong bounce, Lunchtime rally ...",Looking for a strong bounce $SPY -- Lunchtime ...,looking for strong bounce lunchtime rally coming
4,20091246,$PLUG,0.403,[Very intrigued with the technology and growth...,Been doing some work on $PLUG this evening. Ve...,very intrigued with the technology and growth ...
5,5819749,$GMCR,0.000,"[short worked, puts up]",MY $LNKD short worked. Thank you $GMCR for the...,short worked puts
6,709741154393133056,$IBM,-0.296,[overbought],"DJI overbought: $AAPL, $AXP, $BA, $CVX, $IBM, ...",overbought
7,17892972,$JOSB,-0.546,"[absolute garbage still up, stores TOTALLY EMP...",meanwhile this absolute garbage still up. $MW ...,absolute garbage still stores totally empty st...
8,709834259687710720,$CSTM,-0.438,[Biggest Market Losers],2⃣0⃣Biggest Market Losers $VRX $CSTM $ZYNE $EN...,biggest market losers
9,708481442079068160,$PYPL,0.408,[Love this company long time.],$PYPL great vol flow trade here.. Love this co...,love this company long time


In [169]:
test_data['merged_spans'] = merge_spans_test(test_data)

In [170]:
test_data

,cashtag,id,source,spans,text,merged_spans
0,$COST,709723193125175300,twitter,consumers keep cautious stance,U.S. consumers keep cautious stance https://t....,consumers keep cautious stance
1,$CTRP,719529094652698600,twitter,Close $CTRP @46.16 from 43.55 entry +6%;$;+6%,Close $CTRP @46.16 from 43.55 entry +6% #times...,close ctrp from entry
2,$INTC,39048670,stocktwits,Every Reason to Be Bullish,@OphirGottlieb $INTC Every Reason to Be Bullis...,every reason bullish
3,$PANW,37048093,stocktwits,$PANW need anoth $1 to all time high,$PANW need anoth $1 to all time high. $FTNT ne...,panw need anoth all time high
4,$JRCC,6207860,stocktwits,Long setup,$JRCC http://stks.co/1gdx Long setup: One name...,long setup
5,$SBUX,719911771625140200,twitter,will be a solid long term investmen;$;#passive...,"Also added $SBUX today, will be a solid long t...",will solid long term investmen passive also ad...
6,$TSLA,719571433307955200,twitter,recall,$TSLA recall,recall
7,$MAR,711874651043926000,twitter,Marriott wins new Starwood deal by raising cas...,Marriott wins new Starwood deal by raising cas...,marriott wins new starwood deal raising cash p...
8,$ABBV,719559500852662300,twitter,Time To Embrace Dividend Growth,Commented on It's Time To Embrace Dividend Gro...,time embrace dividend growth
9,$MS,709737853639663600,twitter,Bullshit! All #AppleStores are EMPTY in #China...,Bullshit! All #AppleStores are EMPTY in #China...,bullshit all applestores are empty china must ...


## Computing Lougran Lexicon Features

In [ ]:
# Reading dictionary
logran_dict = pd.read_csv('lougran.csv',sep=',')

In [172]:
pos_lougran = []
neg_lougran = []
for index,row in logran_dict.iterrows():
    if row['Positive'] == 2009:
        pos_lougran.append(row['Word'].lower())
    if row['Negative'] == 2009:
        neg_lougran.append(row['Word'].lower()) 

In [173]:
def lex_Lougran():
    words_neg_lougran = []
    fname = 'Loughran_McDonald_AggregateIPOWordList.txt'
    f = open(fname)
    for l in f.readlines():
       
        word = l.strip().lower()
        words_neg_lougran.append(word)
       
    return words_neg_lougran


In [174]:
words_neg_lougran = lex_Lougran()

In [175]:
def lougran_pos_count(text):
    words = text.split()
    pos_count = 0
    for w in words:
        if w.lower() in pos_lougran:
            pos_count = pos_count+1
    return pos_count

def lougran_neg_count(text):
    words = text.split()
    neg_count = 0
    for w in words:
        if w.lower() in words_neg_lougran:
            neg_count = neg_count+1
    return neg_count

In [176]:
training_data['neg_count'] = training_data['merged_spans'].apply(lougran_neg_count)
training_data['pos_count'] = training_data['merged_spans'].apply(lougran_pos_count)

In [177]:
training_data.head()

,id,cashtag,sentiment score,spans,text,merged_spans,neg_count,pos_count
0,719659409228451840,$FB,0.366,[watching for bounce tomorrow],$FB watching for bounce tomorrow,watching for bounce tomorrow,0,0
1,719904304207962112,$LUV,0.638,[record number of passengers served in 2015],record number of passengers served in 2015 #ai...,record number passengers served,0,0
2,5329774,$NFLX,-0.494,[out $NFLX -.35],out $NFLX -.35,out nflx,0,0
3,719891468173844480,$DIA,0.460,"[Looking for a strong bounce, Lunchtime rally ...",Looking for a strong bounce $SPY -- Lunchtime ...,looking for strong bounce lunchtime rally coming,0,1
4,20091246,$PLUG,0.403,[Very intrigued with the technology and growth...,Been doing some work on $PLUG this evening. Ve...,very intrigued with the technology and growth ...,0,0


In [178]:
test_data

,cashtag,id,source,spans,text,merged_spans
0,$COST,709723193125175300,twitter,consumers keep cautious stance,U.S. consumers keep cautious stance https://t....,consumers keep cautious stance
1,$CTRP,719529094652698600,twitter,Close $CTRP @46.16 from 43.55 entry +6%;$;+6%,Close $CTRP @46.16 from 43.55 entry +6% #times...,close ctrp from entry
2,$INTC,39048670,stocktwits,Every Reason to Be Bullish,@OphirGottlieb $INTC Every Reason to Be Bullis...,every reason bullish
3,$PANW,37048093,stocktwits,$PANW need anoth $1 to all time high,$PANW need anoth $1 to all time high. $FTNT ne...,panw need anoth all time high
4,$JRCC,6207860,stocktwits,Long setup,$JRCC http://stks.co/1gdx Long setup: One name...,long setup
5,$SBUX,719911771625140200,twitter,will be a solid long term investmen;$;#passive...,"Also added $SBUX today, will be a solid long t...",will solid long term investmen passive also ad...
6,$TSLA,719571433307955200,twitter,recall,$TSLA recall,recall
7,$MAR,711874651043926000,twitter,Marriott wins new Starwood deal by raising cas...,Marriott wins new Starwood deal by raising cas...,marriott wins new starwood deal raising cash p...
8,$ABBV,719559500852662300,twitter,Time To Embrace Dividend Growth,Commented on It's Time To Embrace Dividend Gro...,time embrace dividend growth
9,$MS,709737853639663600,twitter,Bullshit! All #AppleStores are EMPTY in #China...,Bullshit! All #AppleStores are EMPTY in #China...,bullshit all applestores are empty china must ...


In [179]:
test_data['neg_count'] = test_data['merged_spans'].apply(lougran_neg_count)
test_data['pos_count'] = test_data['merged_spans'].apply(lougran_pos_count)

## Modeling

In [252]:
# Splitting dataset for evaluation purpose
# Later on we replaced X_test with the provided test dataset
X_train, X_test, y_train, y_test = train_test_split(training_data[['id','merged_spans','pos_count','neg_count','stockLex_Pos','stockLex_Neg']],training_data[['sentiment score']], test_size=0.20, random_state=42)

In [350]:
# Computing TF-IDF feature matrix
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5,
                                 stop_words='english')
X_training = vectorizer.fit_transform(X_train.merged_spans)
X_testing = vectorizer.transform(X_test.merged_spans)

In [210]:
# Computing Unigram feature matrix

from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(ngram_range=(1, 1), max_df=0.5,
                                 stop_words='english')
X_trainingGram = vectorizer.fit_transform(X_train.merged_spans)
X_testingGram = vectorizer.transform(X_test.merged_spans)

In [351]:
# Parameters for XGBOOST Regressors
x_params = { 'max_depth':range(3,10,2),
         'n_estimators': [80,100,150,200],
        'subsample':[i/100.0 for i in range(40,90,5)],
        'min_child_weight' :range(1,6,2),
        'colsample_bytree' :[i/100.0 for i in range(40,90,5)],
        'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100],
        'learning_rate':[0.1,0.5],
        'objective':['reg:linear'],
        'scale_pos_weight' : [1,5,10,50,100]
       }
xgb_model = xgb.XGBRegressor()

In [353]:
train_feats =np.column_stack((X_training.toarray(),X_train['pos_count'],X_train['neg_count'],
                              X_train['stockLex_Pos'],X_train['stockLex_Neg']))
test_feats =np.column_stack((X_testing.toarray(),X_test['pos_count'],X_test['neg_count'],
                            X_test['stockLex_Pos'],X_test['stockLex_Neg']))

In [355]:
clf = RandomizedSearchCV(xgb_model, x_params,cv = 3,scoring=cs_scorer)
clf.fit(train_feats,X_train['sentiment score'])


RandomizedSearchCV(cv=3, error_score='raise',
          estimator=XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1),
          fit_params={}, iid=True, n_iter=10, n_jobs=1,
          param_distributions={'n_estimators': [80, 100, 150, 200], 'subsample': [0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85], 'reg_alpha': [1e-05, 0.01, 0.1, 1, 100], 'colsample_bytree': [0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85], 'objective': ['reg:linear'], 'learning_rate': [0.1, 0.5], 'scale_pos_weight': [1, 5, 10, 50, 100], 'max_depth': [3, 5, 7, 9], 'min_child_weight': [1, 3, 5]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score=True, scoring=make_scorer(r2_score),
     

In [356]:
pr = clf.predict(test_feats)

In [358]:
X_test['predicted_score'] = pr

In [347]:
## Evaluation on Validation Set
cosine_similarity(y_test['sentiment score'],y_test['predicted_score'])

array([[ 0.72069268]])

In [367]:
submission = pd.DataFrame({'id':X_test['id'],'cashtag' : X_test['cashtag'],'sentiment score' : X_test['predicted_score']})

In [368]:
submission.head()

,cashtag,id,sentiment score
0,$COST,709723193125175300,-0.350085
1,$CTRP,719529094652698600,0.137532
2,$INTC,39048670,0.380980
3,$PANW,37048093,0.195712
4,$JRCC,6207860,0.549475


In [369]:
submission.to_json('submission.json',orient='records')